In [7]:
import pandas as pd
import numpy as np

# Bestandspaden
FILE_BRIDGES = '/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/raw/BMMS_overview.xlsx'
FILE_ROADS = '/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/Processed/_roads3.csv'

# AANGEPAST: De extensie is veranderd van .csv naar .xlsx
FILE_OUTPUT = '/Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/Processed/BMMS_overview_schoongemaakt_volledig.xlsx'

print("1. Data inladen...")
# We gebruiken read_excel voor het originele bestand, dat is robuuster!
df_bridges = pd.read_excel(FILE_BRIDGES)

# Slim inladen van de CSV om de eerdere 'KeyError: chainage' te voorkomen
df_roads = pd.read_csv(FILE_ROADS, sep=None, engine='python')
df_roads.columns = df_roads.columns.str.strip() # Verwijdert onzichtbare spaties rondom kolomnamen

# ==========================================
# STAP 1: Opgeschoonde wegen-data wiskundig bruikbaar maken
# ==========================================
print("2. Wegen-netwerk voorbereiden...")
# Zet kolommen om naar getallen
df_roads['chainage'] = pd.to_numeric(df_roads['chainage'], errors='coerce')
df_roads['lat'] = pd.to_numeric(df_roads['lat'], errors='coerce')
df_roads['lon'] = pd.to_numeric(df_roads['lon'], errors='coerce')

# Gooi wegen met missende coordinaten of kilometers weg
df_roads = df_roads.dropna(subset=['chainage', 'lat', 'lon'])

# CRUCIAAL: Sorteer op weg en dan oplopend op chainage (vereist voor interpolatie)
df_roads = df_roads.sort_values(by=['road', 'chainage'])

# Verwijder duplicaten: als er 2 metingen zijn op km 1.0, behoud alleen de eerste
df_roads = df_roads.drop_duplicates(subset=['road', 'chainage'], keep='first')

# Maak een dictionary om razendsnel de wegen op te zoeken
road_interpolators = {}
for road, group in df_roads.groupby('road'):
    if len(group) > 1: # We hebben minstens 2 punten nodig voor een lijn
        road_interpolators[road] = (group['chainage'].values, group['lat'].values, group['lon'].values)

# ==========================================
# STAP 2: Bruggen-data filteren
# ==========================================
print("3. Bruggen filteren op missende data...")
df_clean_bridges = df_bridges.dropna(subset=['chainage', 'road']).copy()

# Omdat we een ruwe Excel inladen, checken we even netjes of de kolom bestaat voor we filteren
if 'EstimatedLoc' in df_clean_bridges.columns:
    df_clean_bridges = df_clean_bridges[df_clean_bridges['EstimatedLoc'] != 'error']

# ==========================================
# STAP 3: Bruggen projecteren op de wegen
# ==========================================
print("4. Coördinaten interpoleren op basis van de wegenlijnen...")

def snap_to_road(row):
    road = row['road']
    chainage = pd.to_numeric(row['chainage'], errors='coerce')

    # Huidige label behouden als back-up
    current_loc = row['EstimatedLoc'] if 'EstimatedLoc' in row else ''

    # Als we de weg niet hebben, of de chainage is ongeldig, hou dan het origineel
    if pd.isna(chainage) or road not in road_interpolators:
        return row['lat'], row['lon'], current_loc

    x_chainages, y_lats, y_lons = road_interpolators[road]

    # Check of de brug binnen het bereik van de weg ligt
    if chainage < x_chainages.min() or chainage > x_chainages.max():
        # Brug ligt buiten de gemeten weg, behoud de originele GPS
        return row['lat'], row['lon'], current_loc

    # Bereken het nieuwe punt exact op de lijn
    new_lat = np.interp(chainage, x_chainages, y_lats)
    new_lon = np.interp(chainage, x_chainages, y_lons)

    # We gebruiken de label 'interpolate', net als de docent
    return new_lat, new_lon, 'interpolate'

# Voer de berekening uit voor elke brug
new_coords = df_clean_bridges.apply(snap_to_road, axis=1, result_type='expand')

df_clean_bridges['lat'] = new_coords[0]
df_clean_bridges['lon'] = new_coords[1]
df_clean_bridges['EstimatedLoc'] = new_coords[2]

# ==========================================
# STAP 4: Opslaan naar Processed Folder
# ==========================================
print("5. Klaar! Opslaan als nieuw bestand...")
# AANGEPAST: We gebruiken to_excel in plaats van to_csv
df_clean_bridges.to_excel(FILE_OUTPUT, index=False)
print(f"Resultaat opgeslagen in: {FILE_OUTPUT}")

1. Data inladen...
2. Wegen-netwerk voorbereiden...
3. Bruggen filteren op missende data...
4. Coördinaten interpoleren op basis van de wegenlijnen...
5. Klaar! Opslaan als nieuw bestand...
Resultaat opgeslagen in: /Users/yoran/Documents/Advanced_Simulation/Bangladesh_Assignment/data/Processed/BMMS_overview_schoongemaakt_volledig.xlsx
